# Google Review에서 동반자 유형 빈도수 추출 코드 

* 구글 리뷰 데이터셋 파일들(각 관광지별로 저장된 15개의 파일들)이 있는 폴더를 타겟으로 지정한다. 
* 동반자 유형 사전을 만든다. 
* 구글 리뷰에 대해 특정 조사로 끝나는 단어들만 정제한다. 
* 품사 태깅을 진행하여 명사와 조사를 추출한다. 
* 명사가 동반자 유형 사전에 해당하는지 확인후 카운팅 한다. 
* 빈도수가 높은 순서대로 동반자 유형을 출력한다. 
* 해당 과정을 각 관광지별로 진행하고, 이후 전체 관광지 15개를 대상으로도 진행한다. 

## 타겟 파일 설정

In [1]:
target = 'labeled_spellcheck_8420'

## 사용 라이브러리 

In [2]:
import os # file path 
import glob # file path 
from tqdm import tqdm # execution time
import re # regex
import pandas as pd # dataframe
import copy # deep copy

from collections import Counter
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor

In [3]:
import warnings

warnings.filterwarnings( 'ignore' )

## 경로 설정

In [4]:
!tree -L 3 -N /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420 # jungcheck 폴더내 filtered(전처리 후)의 파일트리 확인

/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420
├── google_reviews_art_culture_complex.csv
├── google_reviews_daecheong_lake.csv
├── google_reviews_dongchundang.csv
├── google_reviews_expo_science_park.csv
├── google_reviews_gyejok_mountain.csv
├── google_reviews_hanbat_arboretum.csv
├── google_reviews_jangtae_mountain.csv
├── google_reviews_observatory.csv
├── google_reviews_oworld_zoo.csv
├── google_reviews_ppuri_park.csv
├── google_reviews_science_museum.csv
├── google_reviews_sungsimdang_bakery.csv
├── google_reviews_uineungjeongi_street.csv
├── google_reviews_water_barrel.csv
└── google_reviews_yuseong_hotspring.csv

0 directories, 15 files


In [5]:
!pwd

/home/aiffel-dj19/jungcheck/DataPreprocessing


In [6]:
g_data_path = '/home/aiffel-dj19/jungcheck/DataPreprocessing'

In [7]:
filtered_path = os.path.join(g_data_path, 'filtered/google_review/')
filtered_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/'

In [8]:
filtered_fds = os.listdir(filtered_path) # fds: 'folders'
print(f'{filtered_path}내의 파일 개수: {len(filtered_fds)}개') 
print()
print(filtered_fds)

/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/내의 파일 개수: 10개

['google_reviews_spacing_8421.csv', 'labeled_spacing_8421', 'spacing_8421', 'google_reviews_labeled_spellcheck_8420.csv', 'labeled_spellcheck_8420', 'spacing_spellcheck_8420', 'spellcheck_8420', 'google_reviews_nolabel_8429.csv', 'google_reviews_labeled_spacing_8421.csv', 'google_reviews_labeled_8429.csv']


In [9]:
# english keyword to korean keyword
eng2kor = {
            'jangtae_mountain': '장태산',
             'gyejok_mountain': '계족산',
             'dongchundang': '동춘당',
             'uineungjeongi_street': '으느정이문화의거리',
             'ppuri_park': '뿌리공원',
             'expo_science_park': '엑스포',
             'sungsimdang_bakery': '성심당',
             'water_barrel': '수통골',
             'yuseong_hotspring': '유성온천',
             'hanbat_arboretum': '한밭수목원',
             'science_museum': '국립중앙과학관',
             'daecheong_lake': '대청호',
             'art_culture_complex': '대전문화예술단지',
             'observatory': '시민천문대',
             'oworld_zoo': '오월드'
}

# korean keyword to english keyword
kor2eng = {v:k for k, v in eng2kor.items()}

print('영어 keyword => 한국어 keyword')
print(eng2kor)
print()
print('한국어 keyword => 영어 keyword')
print(kor2eng)

영어 keyword => 한국어 keyword
{'jangtae_mountain': '장태산', 'gyejok_mountain': '계족산', 'dongchundang': '동춘당', 'uineungjeongi_street': '으느정이문화의거리', 'ppuri_park': '뿌리공원', 'expo_science_park': '엑스포', 'sungsimdang_bakery': '성심당', 'water_barrel': '수통골', 'yuseong_hotspring': '유성온천', 'hanbat_arboretum': '한밭수목원', 'science_museum': '국립중앙과학관', 'daecheong_lake': '대청호', 'art_culture_complex': '대전문화예술단지', 'observatory': '시민천문대', 'oworld_zoo': '오월드'}

한국어 keyword => 영어 keyword
{'장태산': 'jangtae_mountain', '계족산': 'gyejok_mountain', '동춘당': 'dongchundang', '으느정이문화의거리': 'uineungjeongi_street', '뿌리공원': 'ppuri_park', '엑스포': 'expo_science_park', '성심당': 'sungsimdang_bakery', '수통골': 'water_barrel', '유성온천': 'yuseong_hotspring', '한밭수목원': 'hanbat_arboretum', '국립중앙과학관': 'science_museum', '대청호': 'daecheong_lake', '대전문화예술단지': 'art_culture_complex', '시민천문대': 'observatory', '오월드': 'oworld_zoo'}


## 대상 파일 설정

In [10]:
def concatCsv(data_type:str, keyword_kor='계족산'):
    '''
    정제 전 폴더 data와 정제 후 폴더 filtered간의 파일 구성이 따르기 때문에 
    주어진 폴더명에 따라 dataframe을 합해주는 함수
    '''

    if data_type == 'data':
        keyword_path = data_type + '/google_review/' + keyword_kor
        folder_path = os.path.join(g_data_path, keyword_path)
        print(f'folder name: {folder_path}')
        print(f'..: {os.listdir(folder_path)}')
        
        data_list = []
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(f'file path: {file_path}')

            df = pd.read_csv(file_path, encoding='utf-8') 
            search = file.split('.')[0]  # 파일명에서 파일형식(.csv) 제외 후, 검색어 추출             
            df['search'] = search
            df['keyword'] = kor2eng[keyword_kor] 
            data_list.append(df)
            print(f'해당 파일의 데이터 개수: {len(df)}')
            print()
        df = pd.concat(data_list, axis=0)
        print('-'*40)
        
    elif data_type == 'filtered':
        folder_path = os.path.join(g_data_path, data_type + '/google_review/' + target)
        print(f'folder name: {folder_path}')
        
        data_list = []    
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(f'file path: {file_path}')

            df = pd.read_csv(file_path, encoding='utf-8')  # csv 파일 읽기
            print(f'해당 파일의 데이터 개수: {len(df)}')
            print()
            data_list.append(df)
            print('-'*40)
            
        df = pd.concat(data_list, axis=0)
        print(target, '총 데이터 개수: ', len(df))
    return df

##  데이터 불러오기

In [11]:
data_df = concatCsv('filtered', _)

folder name: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_daecheong_lake.csv
해당 파일의 데이터 개수: 491

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_sungsimdang_bakery.csv
해당 파일의 데이터 개수: 1863

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_water_barrel.csv
해당 파일의 데이터 개수: 685

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_observatory.csv
해당 파일의 데이터 개수: 26

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_

## 동반자 유형 사전 만들기 

In [12]:
companions = {
    '가족친지': ['남매', '자매', '형제', '아기', '여보', '아내', '남편', '가족', '친척', '사촌', '부모님', '시부모', '장인어른', '아버님', '어머님', '조카', '이모', '고모', '삼촌', '할아버지', '할머니', '친형', '친언니', '친누나', '친동생', '친오빠', '엄마', '아빠', '딸', '아들', '자식'],
    '친구연인': ['동기', '후배', '선배', '남자친구', '여자친구', '자기', '과동기', '찐친', '절친', '친구', '언니', '동생', '형', '오빠'],
    '직장': ['직장', '회사', '주임님', '회사동료', '사장님', '직장동료', '동료', '직장동기', '회사동기', '회사', '직장선배', '직장후배', '선임', '후임', '부장', '부장님', '차장님', '대리님'],
    '학교': ['학과', '학교', '현장견학', '현장체험', '소풍', '수학여행']
}

In [13]:
companions.keys()

dict_keys(['가족친지', '친구연인', '직장', '학교'])

In [14]:
def dotAdd(sen):
    if sen[-1] in ['다', '요', '죠', '오', '자']:
        sen = sen + '.'
    return sen

## 품사 태깅을 위한 토크나이저 활용
* customized konlpy 활용
    - 장점: 커스텀된 단어의 품사를 추가할 수 있음 

In [15]:
tagger = Twitter()
tagger.add_dictionary(['남매', '자매', '형제', '아기', '가족', '친척', '사촌', '부모님', '시부모', '장인어른', 
                       '아버님', '어머님', '조카', '이모', '고모', '삼촌', '할아버지', '할머니', '친형', '친언니', 
                       '친누나', '친동생', '친오빠', '엄마', '아빠', '딸', '아들', '자식',
                      '동기', '후배', '선배', '남자친구', '여자친구', '자기', '과동기', '찐친', 
                       '절친', '친구', '언니', '동생', '형', '오빠',
                      '직장', '회사', '주임님', '회사동료', '사장님', '직장동료', '직장동기', '회사동기', 
                       '회사', '직장선배', '직장후배', '선임', '후임', '부장', '부장님', '차장님', '대리님',
                      '학과', '학교', '현장견학', '현장체험', '소풍', '수학여행', '주차장'], 'Noun')
tagger.add_dictionary(['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께'], 'Josa')

In [16]:
postprocessor = Postprocessor(
    base_tagger = tagger, # base tagger
#     stopwords = ['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께'], # 해당 단어 필터
#     passwords = passwords, # 해당 단어만 선택
    passtags = {'Noun', 'Josa'}, # 해당 품사만 선택
#     replace = replace, # 해당 단어 set 치환
#     ngrams = ngrams # 해당 복합 단어 set을 한 단어로 결합
)

## 품사태깅 테스트 

In [17]:
for word, pos in postprocessor.pos('아빠하고 아빠의 엄마랑 주차장을 차장님이랑 차장이랑 같이 왔다'):
    print(word, pos)

아빠 Noun
하고 Josa
아빠 Noun
의 Josa
엄마 Noun
랑 Josa
주차장 Noun
을 Josa
차장님 Noun
이랑 Josa
차장 Noun
이랑 Josa
같이 Josa


In [18]:
postprocessor.pos(data_df['comment'].iloc[3123])

[('세', 'Noun'),
 ('이하', 'Noun'),
 ('아기', 'Noun'),
 ('랑', 'Josa'),
 ('가기', 'Noun'),
 ('무', 'Noun')]

In [19]:
'차장' == '주차장' # 주차장을 자꾸 차장으로 인식하는 오류 있음. 데이터 내에 차장과 관련된 데이터 없으므로 사전에서 제거 

False

## 데이터 중 하나로 테스트

In [20]:
print(data_df['comment'].iloc[3123])
for word, pos in postprocessor.pos(data_df['comment'].iloc[3123]):
    for key, value in companions.items():
    #     print(value)
        for v in value: 
            if word == v:
                print(word, '=>', v)

5세 이하 아기랑 가기 넣무 좋아요
아기 => 아기


## 각 관광지별로 동반자 유형 빈도수 추출 

In [21]:
def runAll(keyword):
    # 특정 관광지명을 대상으로 분석 
    test_df = data_df[data_df['keyword'] == keyword]
    test_list = test_df['comment']

    # 각 문장에 마침표 추가 후 한 문단으로 모든 문장을 합해준다. 
    test_list = list(map(dotAdd, test_list))
    test_list = ' '.join(test_list)

    # 
    word_list = [(word, pos) for word, pos in postprocessor.pos(test_list)]

    # 띄어쓰기 기반으로 문단을 한 단어씩 구분
    filtered_words = test_list.split(' ')

    josa = ['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께']
    josa_list = []
    for j in josa:
        for word in filtered_words:
            if j in word and word.endswith(j) and len(word) > len(j):
                josa_list.append(word)

    # 동반자와 같이 올 수 있는 조사 형태가 있는 단어만 추출 
    word_list = " ".join(josa_list) 
    companion_list = []
    for word, _ in postprocessor.pos(word_list):
        for key, value in companions.items():
            for v in value: 
                if word == v:
                    print(word, '=>', key)
                    companion_list.append(key) # 동반자 유형을 할당 
    
    freq = Counter(companion_list)
    print(freq)
    
    if len(freq) != 0:
        companion_type = max(freq.keys(), key=(lambda k: freq[k]))
        print(companion_type)
        print()
    else: 
        print('동반자 유형을 알 수 없습니다.')
        print()
    
    return freq

In [22]:
for keyword in eng2kor.keys():
    key_kor = eng2kor[keyword]
    print('===========================관광지: ', keyword, '(', key_kor, ')', '============')
    runAll(keyword)

===========================관광지:  jangtae_mountain ( 장태산 ) ============
아빠 => 가족친지
아내 => 가족친지
남편 => 가족친지
친구 => 친구연인
친구 => 친구연인
친구 => 친구연인
조카 => 가족친지
가족 => 가족친지
친구 => 친구연인
Counter({'가족친지': 5, '친구연인': 4})
가족친지

===========================관광지:  gyejok_mountain ( 계족산 ) ============
가족 => 가족친지
친구 => 친구연인
가족 => 가족친지
가족 => 가족친지
가족 => 가족친지
Counter({'가족친지': 4, '친구연인': 1})
가족친지

===========================관광지:  dongchundang ( 동춘당 ) ============
아기 => 가족친지
Counter({'가족친지': 1})
가족친지

===========================관광지:  uineungjeongi_street ( 으느정이문화의거리 ) ============
동기 => 친구연인
사장님 => 직장
Counter({'친구연인': 1, '직장': 1})
친구연인

===========================관광지:  ppuri_park ( 뿌리공원 ) ============
아기 => 가족친지
가족 => 가족친지
아기 => 가족친지
아기 => 가족친지
아기 => 가족친지
아내 => 가족친지
친구 => 친구연인
아들 => 가족친지
가족 => 가족친지
가족 => 가족친지
가족 => 가족친지
동생 => 친구연인
가족 => 가족친지
가족 => 가족친지
Counter({'가족친지': 12, '친구연인': 2})
가족친지

===========================관광지:  expo_science_park ( 엑스포 ) ============
여자친구 => 친구연인
가족 => 가족친지
여자친구 => 친구연인
여자친구 => 친구연인
여자친구 

## 관광지 15개 종합하여 동반자 유형 빈도수 추출 

In [23]:
# 모든 관광지 중 방문자 유형 추출
data_list = data_df['comment']

# 각 문장에 마침표 추가 후 한 문단으로 모든 문장을 합해준다. 
data_list = list(map(dotAdd, data_list))
data_list = ' '.join(data_list)

# 
word_list = [(word, pos) for word, pos in postprocessor.pos(data_list)]

# 띄어쓰기 기반으로 문단을 한 단어씩 구분
filtered_words = data_list.split(' ')

josa = ['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께']
josa_list = []
for j in josa:
    for word in filtered_words:
        if j in word and word.endswith(j) and len(word) > len(j):
            josa_list.append(word)

# 동반자와 같이 올 수 있는 조사 형태가 있는 단어만 추출 
word_list = " ".join(josa_list) 
companion_list = []
for word, _ in postprocessor.pos(word_list):
    for key, value in companions.items():
        for v in value: 
            if word == v:
#                 print(word, '=>', key)
                companion_list.append(key)
                
freq = Counter(companion_list)
print(freq)

companion_type = max(freq.keys(), key=(lambda k: freq[k]))
print(companion_type)

Counter({'가족친지': 79, '친구연인': 31, '직장': 5, '학교': 2})
가족친지
